In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="7469b025690441a6900dbdf0fd955211",
                                                           client_secret="94d9a566dcb1458babdfc4992902db7c"))

headers = {
    'Authorization': 'Bearer {token}'.format(token='BQB5Q4pvY0LpcI30Guvah_hgTiYIcVEeboHgDTTgoEs9E7WDTueN1sw2m9_f5zZVpR0Q4-W6_rY2fuQ1Wm3IOabbTG-3Qc2DNt8LcH_OYlr7UpCMiA4kufJ35fSCa6v29aq_LRsxyYwJvMaPPhd1e9vKIub1-QGRLAzUcl1DoF4f85DoeWor9Mszie0HPEOsB5k')
}

import requests
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID from the URI
track_id = '7ouMYWpwJ422jRcDASZB7P'

# actual GET request with proper header
r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)

artist_name = []
track_name = []
track_id = []
track_number = []

lista_response = []
dct_response = {}
# dct_final = {}
for n in range(0, 401, 100):

    results = sp.playlist_tracks(playlist_id='0JiVp7Z0pYKI8diUV6HJyQ', offset= n)

    for i, item in enumerate(results['items']):
        track = item['track']
        artista_nome_str = ''
        for i in range(len(track['artists'])):
            artista_nome_str += track['artists'][i]['name']+", "
        artist_name.append(artista_nome_str[:-1])
        track_name.append(track['name'])
        track_id.append(track['id'])
        if track['id'] == None:
            continue
        # print(track['artists'][0]['name'], track['name'], track['id'])
        r = requests.get(BASE_URL + 'audio-features/' + track['id'], headers=headers)
        lista = r.json()
        dct_response.update({artista_nome_str[:-2]+"^"+track['name']: lista})
    # dct_final.update(dct_response)


In [2]:
df = pd.DataFrame(dct_response).T.reset_index()
df[['artista', 'musica']] = df['index'].str.split('^', expand=True)
df = df.drop("index", axis='columns')

df = df[['artista', 'musica', 'danceability', 'energy', 'key', 'loudness', 'mode',
    'speechiness', 'acousticness', 'instrumentalness', 'liveness',
    'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
    'duration_ms', 'time_signature']]

df = df.rename(columns={
    'artista': 'artista',
    'musica': 'musica',
    'danceability': 'dancabilidade',
    'energy': 'energia', 
    'key':'tonalidade',
    'loudness': 'volume',
    'speechiness': 'presenca_palavras',
    'acousticness': 'acustica', 
    'instrumentalness': 'instrumentalidade',
    'liveness' :'ao_vivo',
    'valence': 'positividade',
    'tempo': 'tempo',
    'duration_ms': 'duracao_ms'
})
df

,artista,musica,dancabilidade,energia,tonalidade,volume,mode,presenca_palavras,acustica,instrumentalidade,ao_vivo,positividade,tempo,type,id,uri,track_href,analysis_url,duracao_ms,time_signature
0,Ed Sheeran,Shape of You,0.825,0.652,1,-3.183,0,0.0802,0.581,0,0.0931,0.931,95.977,audio_features,7qiZfU4dY1lWllzX7mPBI3,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,https://api.spotify.com/v1/audio-analysis/7qiZ...,233713,4
1,The Weeknd,Blinding Lights,0.514,0.73,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4
2,Tones And I,Dance Monkey,0.824,0.588,6,-6.4,0,0.0924,0.692,0.000104,0.149,0.513,98.027,audio_features,2XU0oxnq2qxCpomAAuJY8K,spotify:track:2XU0oxnq2qxCpomAAuJY8K,https://api.spotify.com/v1/tracks/2XU0oxnq2qxC...,https://api.spotify.com/v1/audio-analysis/2XU0...,209438,4
3,"Post Malone, 21 Savage",rockstar (feat. 21 Savage),0.585,0.52,5,-6.136,0,0.0712,0.124,0.00007,0.131,0.129,159.801,audio_features,0e7ipj03S05BNilyu5bRzt,spotify:track:0e7ipj03S05BNilyu5bRzt,https://api.spotify.com/v1/tracks/0e7ipj03S05B...,https://api.spotify.com/v1/audio-analysis/0e7i...,218147,4
4,Lewis Capaldi,Someone You Loved,0.501,0.405,1,-5.679,1,0.0319,0.751,0,0.105,0.446,109.891,audio_features,7qEHsqek33rTcFNT9PFqLf,spotify:track:7qEHsqek33rTcFNT9PFqLf,https://api.spotify.com/v1/tracks/7qEHsqek33rT...,https://api.spotify.com/v1/audio-analysis/7qEH...,182161,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Red Hot Chili Peppers,Snow (Hey Oh),0.427,0.9,11,-3.674,1,0.0499,0.116,0.000017,0.119,0.599,104.655,audio_features,2aibwv5hGXSgw7Yru8IYTO,spotify:track:2aibwv5hGXSgw7Yru8IYTO,https://api.spotify.com/v1/tracks/2aibwv5hGXSg...,https://api.spotify.com/v1/audio-analysis/2aib...,334667,4
496,Surfaces,Sunday Best,0.878,0.525,5,-6.832,1,0.0578,0.183,0,0.0714,0.694,112.022,audio_features,1Cv1YLb4q0RzL6pybtaMLo,spotify:track:1Cv1YLb4q0RzL6pybtaMLo,https://api.spotify.com/v1/tracks/1Cv1YLb4q0Rz...,https://api.spotify.com/v1/audio-analysis/1Cv1...,158571,4
497,"Kygo, Parson James",Stole the Show,0.679,0.662,8,-6.069,0,0.174,0.155,0,0.152,0.438,99.879,audio_features,4iZPNYqzI2L0uwuUKun7Aa,spotify:track:4iZPNYqzI2L0uwuUKun7Aa,https://api.spotify.com/v1/tracks/4iZPNYqzI2L0...,https://api.spotify.com/v1/audio-analysis/4iZP...,222801,4
498,GAYLE,abcdefu,0.695,0.54,4,-5.692,1,0.0493,0.299,0,0.367,0.415,121.932,audio_features,4fouWK6XVHhzl78KzQ1UjL,spotify:track:4fouWK6XVHhzl78KzQ1UjL,https://api.spotify.com/v1/tracks/4fouWK6XVHhz...,https://api.spotify.com/v1/audio-analysis/4fou...,168602,4


In [3]:
# for id_musica in df['id']:
#     generos_str = ''
#     artista_nome_str = ''
#     if type(id_musica) == float:
#         continue
#     for artista in sp.track(id_musica)['artists']:
#         id_artista = artista['uri'].split(":")[-1]
#         nome_artista = artista['name']
#         genero = (sp.artist(id_artista)['genres'])
#         generos_str += ", ".join(genero)+", "
#         artista_nome_str += nome_artista+", "
#     df.loc[df['artista'] == artista_nome_str[:-2], 'genero'] = generos_str[:-2]

In [4]:
df = df.drop(["type", "uri", "track_href", "analysis_url", "time_signature", "id", "mode"], axis='columns')

In [5]:
df

,artista,musica,dancabilidade,energia,tonalidade,volume,presenca_palavras,acustica,instrumentalidade,ao_vivo,positividade,tempo,duracao_ms
0,Ed Sheeran,Shape of You,0.825,0.652,1,-3.183,0.0802,0.581,0,0.0931,0.931,95.977,233713
1,The Weeknd,Blinding Lights,0.514,0.73,1,-5.934,0.0598,0.00146,0.000095,0.0897,0.334,171.005,200040
2,Tones And I,Dance Monkey,0.824,0.588,6,-6.4,0.0924,0.692,0.000104,0.149,0.513,98.027,209438
3,"Post Malone, 21 Savage",rockstar (feat. 21 Savage),0.585,0.52,5,-6.136,0.0712,0.124,0.00007,0.131,0.129,159.801,218147
4,Lewis Capaldi,Someone You Loved,0.501,0.405,1,-5.679,0.0319,0.751,0,0.105,0.446,109.891,182161
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Red Hot Chili Peppers,Snow (Hey Oh),0.427,0.9,11,-3.674,0.0499,0.116,0.000017,0.119,0.599,104.655,334667
496,Surfaces,Sunday Best,0.878,0.525,5,-6.832,0.0578,0.183,0,0.0714,0.694,112.022,158571
497,"Kygo, Parson James",Stole the Show,0.679,0.662,8,-6.069,0.174,0.155,0,0.152,0.438,99.879,222801
498,GAYLE,abcdefu,0.695,0.54,4,-5.692,0.0493,0.299,0,0.367,0.415,121.932,168602


In [6]:
df.to_excel("databaseNovo.xlsx")